In [1]:
from copy import deepcopy

import numpy as np

In [2]:
from build123d import *

In [3]:
# you can skip this cell if you're not using vs code with the ocp viewer extension
from ocp_vscode import *
set_port(3939)

In [4]:
waffle_shrink_ratio = .8
waffle_height_ratio = 1.2
bottom = Rectangle(1, 1)
top = Rectangle(waffle_shrink_ratio, waffle_shrink_ratio)
waffle_nub = loft([bottom, Pos(0,0, waffle_height_ratio)*top])

show(waffle_nub)

+

In [5]:
top_fillet = .2
side_fillet = .1

waffle_nub = fillet(waffle_nub.edges().group_by()[-1], top_fillet)
waffle_nub = fillet(waffle_nub.edges().group_by()[1], side_fillet)

show(waffle_nub)


+

In [6]:
iron_height = 20
iron_width = 80

waffle_half = Box(iron_width, iron_width, iron_height)

show(waffle_half)

+

In [7]:
waffle_grid_n = 5
empty_fraction = .2
nub_height_factor = .7

nub_spacing = iron_width / waffle_grid_n

empty_spacing = empty_fraction*nub_spacing
nub_scale = iron_width/ waffle_grid_n - empty_spacing
rescaled_nub = scale(waffle_nub, (nub_scale, nub_scale, nub_height_factor * iron_height/waffle_height_ratio))

In [8]:
np.linspace(-1, 1, 5)

array([-1. , -0.5,  0. ,  0.5,  1. ])

In [9]:
waffle_w_nubs = deepcopy(waffle_half)

for i in np.linspace(-1, 1, waffle_grid_n):
    x = i * (iron_width-nub_spacing)/2
    for j in np.linspace(-1, 1, waffle_grid_n):
        y = j * (iron_width-nub_spacing)/2

        waffle_w_nubs += Pos(x, y, iron_height/2) * rescaled_nub
        
show(waffle_w_nubs)

+

In [10]:
outer_edge_width = 5

outer_edge_box_height = iron_height + nub_height_factor * iron_height * .9
outer_edge_box = Box(iron_width + outer_edge_width*2, 
                    iron_width + outer_edge_width*2, 
                    outer_edge_box_height)
outer_edge_cut_box = Box(iron_width, 
                         iron_width, 
                         outer_edge_box_height)
outer_edge_box = outer_edge_box - outer_edge_cut_box
show(outer_edge_box)

+

In [11]:
waffle_boxed = waffle_w_nubs + Pos(0, 0, (outer_edge_box_height - iron_height)/2)*outer_edge_box
show(waffle_boxed)

+

In [12]:
egg_height = 25

eggface = make_face(ThreePointArc((0,0,0), (0,egg_height/3,egg_height/3), (0,0,egg_height)))
egg = revolve(eggface)
show(egg)

+

In [13]:
waffle_egged = waffle_boxed + Pos(-iron_width/4, -iron_width/4, iron_height/3)*egg

show(waffle_egged)

+

Sealion is from https://www.thingiverse.com/thing:1092482 - native file doesn't work but loading in freecad and then re-exporting works. But note the sealion import takes ~10 min

In [24]:
importer = Mesher()
sealion = importer.read("SeaLionFlatUnderside-reexp.stl")[0]

In [30]:
offset = sealion.center()
offset.X *= -1
offset.Y *= -1
offset.Z = -10
centered_sealion = Pos(offset)*sealion

By-eye inspection shows the base is at 10mm, although the other dimensions are a bit confusingly centered.

In [45]:
sealion_size = 50
rescaled_sealion = centered_sealion.scale(sealion_size/max(centered_sealion.bounding_box().size))

In [52]:
adjusted_sealion = Pos(iron_width/5, iron_width/5, iron_height/2) * Rot(Z=135) * rescaled_sealion

In [53]:
waffle_sealion = waffle_egged + adjusted_sealion

/home/erik/3dprinting/hthv4_logo3d/.pixi/envs/default/lib/python3.12/site-packages/build123d/topology.py:1784: UserWarning: Unable to clean Compound at 0x725af309ed80, label(), #children(0)
  warnings.warn(f"Unable to clean {self}")


In [54]:
show(waffle_sealion)

+

In [55]:
export_stl(waffle_sealion, "waffle_sealion.stl")

True